# En este Notebook realizaremos Análisis de casos completos e Imputación por la media/mediana/moda, valor arbitrario, categoría dedicada, imputación aleatoria e indicador binario.


In [ ]:
import pandas as pd
# para mostrar todas las columnas del dataframe en el notebook
pd.set_option('display.max_columns', None)

# Para graficado
import matplotlib.pyplot as plt
import seaborn as sns

# Para divisdir el conjunto de datos
from sklearn.model_selection import train_test_split

# Para la imputación de datos con sklearn
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

In [ ]:
# Cargando datos
data = pd.read_csv('../../datasets/raw/credict_approval.csv')
data.head()

In [ ]:
# Número de variables y objetos en el dataset
data.shape

In [ ]:
# Tipos de datos
print(data.dtypes,'\n')
print(data.dtypes.value_counts())

In [ ]:
# Variables con valores faltantes
print(data.isnull().any())

In [ ]:
(
       data
       .isnull()
       .sum()
       .sort_values(ascending=False)
)

In [ ]:
# Inspeccionemos el porcentaje de valores faltantes en cada variable
data.isnull().mean().sort_values(ascending=False)

In [ ]:
#Gráfica de Proporción de valores nulos por variable
(
    data
    .isnull()
    .melt(value_name='missing')
    .pipe(
        lambda df: (
            sns.displot(
                data=df,
                y='variable',
                hue='missing',
                multiple='fill',
                aspect=2
            )
        )
    )
)

In [ ]:
#valores nulos en todo el conjunto de datos
(
    data
    .isnull()
    .pipe(
        lambda df: sns.heatmap(data=df)
    )
)

In [ ]:
# Separamos las Variables con datos faltantes
missin_value_vars = ['A14','A1','A2','A6','A7','A4','A5']
numerical_features_with_missing_values = ['A2','A14']
non_numerical_features_with_missing_values = ['A1','A6','A7','A4','A5']

### Análisis de Casos Completos

In [ ]:
# Creamos un dataset con casos completos
data_cca = data.dropna()
print('Número total de observaciones: {}'.format(len(data)))
print('Número de observaciones con casos completos: {}'.format(len(data_cca)))

In [ ]:
# Tambien podemos indicar qué variables a considerar con casos completos
data_cca = data.dropna(subset=numerical_features_with_missing_values)
print('Number of total observations: {}'.format(len(data)))
print('Number of observations with complete cases: {}'.format(len(data_cca)))

### Imputación por la media/mediana/moda utilizando Scikit-learn

In [ ]:
# Separemos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    data[numerical_features_with_missing_values],
    data['A16'],
    test_size=0.3,
    random_state=0)

In [ ]:
# Instanciamos un objeto de clase SimpleImputer
imputer = SimpleImputer(strategy='median')

# Entrenemos al imputador sobre los datos de entrenamiento (aprenderá la mediana de todas las variables)
imputer.fit(X_train)

# Observemos las medianas aprendidas:
imputer.statistics_

In [ ]:
# Realizamos imputacion en los conjuntos de entrenamiento y prueba
# NOTA: Los datos se regresan como numpy arrays!

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# Corroboramos que los valores faltantes fueron imputados
pd.DataFrame(X_train).isnull().sum()

### Imputación con la moda utilizando sklearn

In [ ]:
# Separamos los datos en entrenamiento y prueba (variable categóricas)

X_train, X_test, y_train, y_test = train_test_split(
    data[non_numerical_features_with_missing_values], data['A16'], test_size=0.3, random_state=0)

In [ ]:
# creamos el objeto de SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# entrenamos
imputer.fit(X_train)

# observamos las modas aprendidas:
imputer.statistics_

In [ ]:
# Imputamos en los conjuntos de entrenamiento y pruaba y verificamos la imputación
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

pd.DataFrame(X_train).isnull().sum()

### Imputación con Valor arbitrario usando sklearn

In [ ]:
# Separamos los datos en ctos. de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data[numerical_features_with_missing_values],
    data['A16'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

In [ ]:
# Instanciamos y entrenamos el SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value=99)
imputer.fit(X_train)

imputer.statistics_

In [ ]:
# Imputamos los conjuntos de entrenamiento y prueba (numpy arrays)

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# revisamos que los valores faltantes fueron imputados
pd.DataFrame(X_train).isnull().sum()

In [ ]:
# Si quisiéramos continuar con nuestro análisis de datos, tendríamos que codificar X_train en un dataframe:
X_train = pd.DataFrame(
    X_train,
    columns=imputer.get_feature_names_out(),  # the variable names
)

X_train.head(10)

In [ ]:
# Exploremos los cambios en la distribución despues de la imputacion

X_train.hist(bins=50, figsize=(15, 5))
plt.show()

### Imputación con categoría dedicada usando sklearn

In [ ]:
# Separemos los datos en entrenamiento y prueba descritos sólo por las variables no numéricas

X_train, X_test, y_train, y_test = train_test_split(
    data[non_numerical_features_with_missing_values], data['A16'], test_size=0.3, random_state=0)

In [ ]:
# Creamos una instancia de SimpleImputer y entrenamos
imputer = SimpleImputer(strategy='constant', fill_value='Missing')
imputer.fit(X_train)

# Vemos estadisticas:
imputer.statistics_

In [ ]:
#Realizamos la imputacion

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

pd.DataFrame(X_train).isnull().sum()

In [ ]:
#Convertimos a un dataframe
X_train = pd.DataFrame(
    X_train,
    columns=imputer.get_feature_names_out(),  # the variable names
)

X_train.head(10)

### Imputación Aleatoria

Se deja de ejercicio 

### Agregando un indicador binario

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

In [ ]:
indicator = MissingIndicator(error_on_new=True, features='missing-only')
indicator.fit(X_train) 

# Podemos ver las variables con NaNs:
# Los resultados muestran los índices de las columnas del numpy array

indicator.features_

In [ ]:
def get_missing_value_indicatord_dataset(dataset, trained_indicator):

    # Creamos un nombre de columna para cada variable con valores faltantes
    indicator_cols = [c+'_NA' for c in dataset.columns[trained_indicator.features_]]

    # Concatenamos el conjunto de datos original con los indicadores de valores faltantes
    dataset = pd.concat([
    dataset.reset_index(),
    pd.DataFrame(trained_indicator.transform(dataset), columns = indicator_cols)],
    axis=1)

    return dataset

In [ ]:
indicatedData= get_missing_value_indicatord_dataset(X_train,indicator)
indicatedData.head(100)

In [ ]:
# Hacemos lo mismo para el cto. de prueba
tmp = indicator.transform(X_test)

X_test = pd.concat(
    [
        X_test.reset_index(),
        pd.DataFrame(tmp, columns=indicator.get_feature_names_out()),
    ],
    axis=1,
)

X_test.head()

### Imputación por la media/moda + indicador binario

Se deja como ejercicio